In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [3]:
# Load Dataset
df = pd.read_csv('../../df.csv')

In [4]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [5]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [6]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [7]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks. 
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [8]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [9]:
#Set the DATETIME for fecha 
DATETIME = df.index.values[::29]

In [10]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day 
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [11]:
# Select individual stocks and remove demographics
aapl = df.iloc[::29  ,:74]
f    = df.iloc[10::29,:74]
nvda = df.iloc[22::29,:74]
tsla = df.iloc[25::29,:74]
wmt  = df.iloc[27::29,:74]

In [12]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * 5  # 24 time periods per day per stock
total_rows = len(aapl['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(aapl['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_1 = Sequential()
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))
model_1.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_1.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_1.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

aapl_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = aapl.iloc[i:train_rows + i, 1:] 
    test   = aapl.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_1.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_1.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    aapl_result = pd.concat([aapl_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 23338.8594
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 23068.5410
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 22660.7793
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 22017.8301
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 20995.3672
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 19480.9238
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 17376.0371
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 14635.6865
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 11414.7041
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 8004.5645
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 4796.7847
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 2349.6423
Epoch 13/200
6/6

In [13]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * 5  # 24 time periods per day per stock
total_rows = len(f['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(f['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_2 = Sequential()
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))
model_2.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_2.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_2.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

f_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = f.iloc[i:train_rows + i, 1:] 
    test   = f.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_2.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_2.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    f_result = pd.concat([f_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 179.5450
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 158.8222
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 133.4080
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 99.5601
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 57.7351
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 20.1386
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 3.9147
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 3.1890
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 2.8149
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 2.2267
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 2.2139
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 2.0280
Epoch 13/200
6/6 [==============================] - 

6/6 [==============================] - 0s 3ms/step - loss: 1.1168
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2182
Epoch 7: early stopping
Count Down: 0


In [14]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * 5  # 24 time periods per day per stock
total_rows = len(nvda['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(nvda['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_3 = Sequential()
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))
model_3.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_3.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_3.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

nvda_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = nvda.iloc[i:train_rows + i, 1:] 
    test   = nvda.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_3.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_3.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    nvda_result = pd.concat([nvda_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 29095.5723
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 28806.1270
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 28375.4375
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 27652.2832
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 26470.9004
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 24690.0703
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 22143.4277
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 18803.9961
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 14876.7510
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 10639.5195
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 6606.2515
Epoch 12/200
6/6 [==============================] - 0s 3ms/step - loss: 3389.3105
Epoch 13/200
6/

6/6 [==============================] - 0s 3ms/step - loss: 164.7922
Epoch 15/200
6/6 [==============================] - 0s 3ms/step - loss: 171.0758
Epoch 16/200
6/6 [==============================] - 0s 3ms/step - loss: 165.9378
Epoch 16: early stopping
Count Down: 11
Epoch 1/200
6/6 [==============================] - 0s 3ms/step - loss: 165.3182
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 162.7847
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 163.0697
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 160.1264
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 160.7456
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 165.9982
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 155.1736
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 150.7955
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 150.1750
Epoch 10/2

In [15]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * 5  # 24 time periods per day per stock
total_rows = len(tsla['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(tsla['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_4 = Sequential()
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))
model_4.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_4.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_4.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

tsla_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = tsla.iloc[i:train_rows + i, 1:] 
    test   = tsla.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_4.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_4.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    tsla_result = pd.concat([tsla_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 69275.6953
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 68947.4766
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 68516.8359
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 67833.7188
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 66673.1250
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 64798.2344
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 61936.2305
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 57892.8555
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 52427.5273
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 45635.4414
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 37844.6289
Epoch 12/200
6/6 [==============================] - 0s 3ms/step - loss: 29586.8496
Epoch 13/200


6/6 [==============================] - 0s 3ms/step - loss: 550.1975
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 535.0322
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 514.1691
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 522.5949
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 517.9814
Epoch 12/200
6/6 [==============================] - 0s 3ms/step - loss: 503.8064
Epoch 13/200
6/6 [==============================] - 0s 3ms/step - loss: 544.7415
Epoch 14/200
6/6 [==============================] - 0s 3ms/step - loss: 487.2735
Epoch 15/200
6/6 [==============================] - 0s 3ms/step - loss: 505.5602
Epoch 16/200
6/6 [==============================] - 0s 3ms/step - loss: 534.8079
Epoch 17/200
6/6 [==============================] - 0s 3ms/step - loss: 507.6623
Epoch 18/200
6/6 [==============================] - 0s 3ms/step - loss: 502.0355
Epoch 18: early stopping
Count Down: 3
Epoc

In [16]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * 5  # 24 time periods per day per stock
total_rows = len(wmt['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(wmt['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_5 = Sequential()
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))
model_5.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_5.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_5.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

wmt_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = wmt.iloc[i:train_rows + i, 1:] 
    test   = wmt.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_5.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_5.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    wmt_result = pd.concat([wmt_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 16268.7422
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 16043.3037
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 15721.0352
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 15210.5107
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 14415.4697
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 13245.3643
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 11592.6455
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 9508.2725
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 7128.9956
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 4702.0620
Epoch 11/200
6/6 [==============================] - 0s 3ms/step - loss: 2569.4114
Epoch 12/200
6/6 [==============================] - 0s 3ms/step - loss: 1119.5746
Epoch 13/200
6/6 [

6/6 [==============================] - 0s 3ms/step - loss: 133.5124
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 128.9157
Epoch 9: early stopping
Count Down: 0


In [17]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [18]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [19]:
new_rows = []
stock_result = aapl_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['AAPL',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,9.64181,4.895383,4.261478,0.393937


In [20]:
new_rows = []
stock_result = f_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['F',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,F,0.968666,5.483406,0.429655,0.423006


In [21]:
new_rows = []
stock_result = nvda_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['NVDA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,NVDA,13.384757,7.810694,6.213166,0.50203


In [22]:
new_rows = []
stock_result = tsla_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['TSLA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,TSLA,35.52258,14.661574,21.319619,0.460251


In [23]:
new_rows = []
stock_result = wmt_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['WMT',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,WMT,9.600499,5.318621,1.60342,0.407181
